# OCR For Text Extraction
---
## 1. Installing Packages & Import Modules
Packages required to do text extraction are:
- **TensorFlow**
: Text detection and text recognition model builder
- **PyPDF2**
: PDF document handler, capable of retrieve text and metadata from PDFs
- **DocTR**
: OCR logic handler, capable for parsing textual information from document or images

In [3]:
# Main Libs
!pip install tensorflow
!pip install PyPDF2
!pip install python-doctr

# Supporting Libs
!pip install python-doctr[tf]
!pip install python-doctr[torch]
!pip install rapidfuzz==2.15.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.5 MB/s eta 0:00:00
  Attempting uninstall: rapidfuzz
    Found existing installation: rapidfuzz 3.1.1
    Uninstalling rapidfuzz-3.1.1:
      Successfully uninstalled rapidfuzz-3.1.1


**UserWarning** \
If warning message pops up, just ignore it.

In [1]:
import tensorflow as tf
import PyPDF2 as pdf
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## 2. Building Models
So to make OCR works, we need to combine text detection model and text recognition model as an OCR pipeline to recognize text characters.

### Text Detection Model
For the text recognition model, we will use **DB ResNet50**. From the sources that I have read, this model somehow very popular among other document OCR apps for text detection model. It's also recommended by DocTR to use this model instead of the other models. Further research may be needed to understand why this is used.

### Text Recognition Model
For the text recognition model, we will use **CRNN VGG-16 Backbone**. Same as ResNet50, this model somehow very popular among other OCR apps for text recognition model. It's also recommended by DocTR to use this model instead of the other models. Further research may needed to understand why this is used.

**Status:** \
Combining both of these models is somewhat complicated. I have built the models from my local environment, still have no idea how to export it for passing DocTR ocr_predictor arguments. Right now, we will use pretrained model from DocTR itself so it doesn't crash while passing ocr_predictor args. The result from self trained model and pretrained model are nearly the same anyway, so don't worry about it.

In [ ]:
# this block of code will be updated as soon as my self trained models works properly

## 3. Extract Text using Trained OCR Models with DocTR
To extract text with DocTR, we have to use built in ocr_predictor function from DocTR. This function allow us to use self trained or pretrained models for text detection and text recognition. The ocr_predictor returns a document object with a nested structure (with Page, Block, Line, Word, Artefact).

In [2]:
def extract_with_ocr(file):
    # pretrained model
    model = ocr_predictor(det_arch='db_resnet50', reco_arch='crnn_vgg16_bn', pretrained=True)

    # reading files
    document = DocumentFile.from_pdf(file)

    # analyze
    result = model(document)

    # export to json
    output = result.export()

    # grouping detected words
    separated_words = []
    for page in output["pages"]:
        for block in page["blocks"]:
            for line in block["lines"]:
                for word in line["words"]:
                    separated_words.append(word["value"])
    
    # combining separated words into sentences
    output = " ".join(separated_words)
    
    return output

The function below is needed to check if the pdf file is a scanned pdf (image based) or digital pdf (text based). If the function didn't detect any characters, it will be considered scanned pdf, so it had to handle extracting text using OCR. If the function detected some characters (100 chars at least) in the pdf file, then it will be considered digital pdf, so it just need to extract text directly from PyPDF2 for better accuracy.

This function return strings of extracted text from the pdf file.

In [3]:
import re

def text_extractor(file):
    # read file
    reader = pdf.PdfReader(file)
    page = reader.pages[0]
    
    # checks for text
    # if chars detected more than 100 chars, it will be considered scanned pdf
    if len(page.extract_text()) > 100: # tweak this number for character count if you want
        # extract text
        text = page.extract_text()
        
        # clean text
        clean_text = text.lower()

        # remove symbols
        clean_text = re.sub(r'[^\w\ \n]', '', clean_text)
        
        return clean_text
    else:
        return extract_with_ocr(file)

## 4. Let's Test It
This funtion will print out the extracted text from your document. You can play around with any pdf file you want to extract text from. Just change the file_name value to your pdf document path.

In [4]:
if __name__ == "__main__":
    # test scanned pdf
    print("Scanned PDF extracted text example:")
    file_name_2 = "example_scanned.pdf" # select path to your pdf document in your local environment
    print(text_extractor(file_name_2))

Scanned PDF extracted text example:


  0%|          | 0/94178964 [00:00<?, ?it/s]

  0%|          | 0/58758994 [00:00<?, ?it/s]

Jane Franklin Douglas-Kelly Professor of English Address University of Pennsylvania Philadelphia, PA 19104, USA Home Address: 713 N 4th St, Philadelphia, PA 19123, USA Phone 267-861-5317 E-mail jane_rankin@gmail.com LinkedIn Inkedin.cominjanefrandlin Twitter twiercom/janemanefmnan Proactive lecturer and researcher with 10+ years of experience teaching courses on undergraduate and postgraduate levels. Supervised 22 BA theses, 9 MA theses, and 2 PhD dissertations. Edited and co-authored 3 monographs on contemporary trends in drama and performative arts. Published over 30 articles in peer-reviewed journals. Education 2000-08 - 2008-05 University of Chicago, Chicago, IL, PhD in Comparative Literature The Department of Modern Laguages Thesis supervisor: Professor George Swan The Department of English Graduated summa cum laude Ideas and Values of the Movement" Thesis supervisor: Professor Anne Plath Academic achievements: Dean's List 2000 Thesis title: "The Lesbianism of Marcel Proust: An An

---
# Summary
So, this OCR file works just fine. The OCR models predict the words with good accuracy too. But, those models used comes from DocTR pretrained model, so they are already guaranteed that the result of text extraction will be great. Self trained models will be used if they can work properly with DocTR ocr_predictor function args. Some researches may be conducted for reevaluating and remodeling to updgrade and boost accuracy for OCR model.

In [ ]:
# Thanks